In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false
}

# MD with openMM - python API
This jupyter notebook shows how to run a Molecular dynamics (MD) simulations using the openMM python package. First of all we import the openMM packages that we need for the simulations, plus some general one for handling the output.

In [ ]:
########################## import openMM ###############################
import openmm as mm
from openmm.app import *
from openmm.unit import *
from openmmtools import integrators
from openmmtools import integrators as mmt
    
########################## File handling libraries #####################
import subprocess
from sys import stdout
import numpy as np

########################## Initialise random seeds #####################
import time
import random
# random.seed(123456) # <- use this for reproducibility
random.seed(int(time.time()))

We can now create our __system__. The dimensions of the system and atoms' coordinates are read from a _PDB_ file. In this case all the atoms are Argon atoms, so we can easily add them to the __system__

In [ ]:
pdb    = PDBFile('mix.pdb')
box    = pdb.topology.getPeriodicBoxVectors()

system = mm.System()
system.setDefaultPeriodicBoxVectors(*box)

## Model binary mixture
In this example we want to simulate a binary mixture _soft-spheres_, which can illustrate how intermolecular forces determine the mixing behaviour of simple fluid. Soft-spheres can also provide a realistic representation of colloidal particles in solution.

The interaction energy between _soft-spheres_ is normally described using the Lennard-Jones function

\begin{equation}
U_{ij}(r) = 4\varepsilon_{ij}\Big[ \Big(\frac{\sigma_{ij}}{r}\Big)^{12} - \Big(\frac{\sigma_{ij}}{r}\Big)^6 \Big]
\end{equation}

where $i$ and $j$ indicate the particle types, $r_{ij}$ is their distance, $\varepsilon_{ij}$ and $\sigma_{ij}$ are parameters that define the strength of the interaction.
$\varepsilon_{ij}$ is the depth of the energy well and $\sigma_{ij}$ is related to the size of the particles.

Although these particles are not representative of atomic systems, for visualisation purposes it is convenient to give name them after some elements. In this way our spherical particles will also be automatically coloured and assigned a size. It is also convenient to define the interaction parameters that are representative of a real system, which would allow us to make a simple connection between the simulation results and the _real_ time, energy and temperature scales. In this case we chose the interaction parameters for Argon, but we called the atoms "Nitrogen" and "Oxygen", which are normally coloured in blue and red, respectively.

In [ ]:
atomForceField = []
atomForceField.append({
    "type" : "N",
    "mass" : 30. * amu,
    "sigma" :  0.340,
    "epsilon" : 1.0,
})
atomForceField.append({
    "type" : "O",
    "mass" : 30. * amu,
    "sigma" :  0.340,
    "epsilon" : 1.0,
})
numAtomTypes = len(atomForceField)

OpenMM has some optimised functions that can be used to define custom interactions between particles. The class __CustomNonbondedForce__ takes one argument, which contains the expression for the energy and the _rules_ to compute the interaction parameters between the particles.
Normally this is done using the Lorentz-Berthelot mixing rules, but it is also possible to define $(N \times N)$ interaction matrices for the parameters, where $N$ is the number of unique elements in the system.

In [ ]:
expression = '4*eps*( (sig/r)^12 - (sig/r)^6 ); \
              eps=epsilon(type1, type2); \
              sig=sigma(type1, type2)'

We can now build the interaction matrices for the $\varepsilon$ and $\sigma$ parameters, using the Lorentz-Berthelot mixing rules.

In [ ]:
eii = atomForceField[0]["epsilon"]
sii = atomForceField[0]["sigma"]
ejj = atomForceField[1]["epsilon"]
sjj = atomForceField[1]["sigma"]

eij = 1.0
sij = 0.5 * (sii + sjj)

# Interaction matrices
epsilon_r = np.array([ [eii, eij], 
                       [eij, ejj], ], dtype="float64")
sigma_r = np.array([ [sii, sij],
                     [sij, sjj], ], dtype="float64")

# The matrices are not converted to list to be fed to openMM
epsilonLST_r = (epsilon_r).ravel().tolist()
sigmaLST_r   = (sigma_r).ravel().tolist()

We can now create the __CustomNonbondedForce__ object.

In [ ]:
# Creation of the force object
LJ = mm.CustomNonbondedForce(expression)
LJ.setNonbondedMethod(mm.NonbondedForce.CutoffPeriodic)
LJ.setCutoffDistance(1.5*nanometer)

# Function for the interaction matrices for the parameters
LJ.addTabulatedFunction(
    'epsilon', mm.Discrete2DFunction(numAtomTypes, numAtomTypes, epsilonLST_r))
LJ.addTabulatedFunction(
    'sigma', mm.Discrete2DFunction(numAtomTypes, numAtomTypes, sigmaLST_r))

# Only one paramaters per particle is required since the actual
# values of epsilon and sigma are taken from the matrices
_ = LJ.addPerParticleParameter('type')

We now add the particles from the PDB file into the __system__ and add them to the __CustomNonbondedForce__ object.

In [ ]:
numAtoms = system.getNumParticles()
for i in pdb.topology.atoms():
    iType = next(
        (t for t,d in enumerate(atomForceField) if d['type'] == i.name), None)
    mass = atomForceField[iType]["mass"]
    system.addParticle( mass )
    LJ.addParticle( [iType] )

LJ.setForceGroup(system.getNumForces()+1)
_ = system.addForce(LJ)

Now that we have created a system with the particles and their interactions, we can start working on defining the simulation that we want to run by setting the thermodynamic ensemble.

1. NVE - microcanonical, _i.e._ constant number of atoms (N), Volume and Energy
2. NVT - canonical,  _i.e._ constant number of atoms (N), Volume and Temperature
3. NPH - isobaric-isoenthalpic, _i.e._ constant number of atoms (N), Pressure and entHalpy
4. NPT - isothermal-isobaric,  _i.e._ constant number of atoms (N), Pressure and Temperature

For the constant pressure simulations there are also various options for how to control the shape of the simulation cell. In this case we can choose between isotropic or orthorhombic deformations of the system.

We also need to define the simulation temperature and pressure, the timestep and a few other parameters.

In [ ]:
########################## Simulation parameters ##########################
minimise    = False             # <-- perform energy minimisation
NVE         = False             # <-- MD with no thermostat (NVE)
NPT_iso     = True              # <-- montecarlo barostat isotropic
NPT_ort     = False             # <-- montecarlo barostat orthorhombic

timestep    = 0.005*picoseconds # <-- MD timestep
nsteps      = 2000              # <-- total number of timesteps
ntraj       = 100               # <-- frequency of trajectory output
nthermo     = 100               # <-- frequency of data output

temperature = 80*kelvin        # <-- temperature
pressure    = 1*atmosphere      # <-- external pressure

trel        = 1/picoseconds     # <-- thermostat relaxation time
nupdt       = 25                # <-- how often the volume is updated

According to the ensemble chosen, we add different integrators to the __system__.

In [ ]:
########################## Set integrator #################################
if NVE:
    integrator = mmt.VelocityVerletIntegrator( timestep )
else:
    integrator = mmt.LangevinIntegrator( temperature , trel , timestep )

# Barostat
if NPT_iso:
    system.addForce(mm.MonteCarloBarostat( pressure , temperature , nupdt ))
if NPT_ort:
    system.addForce(mm.MonteCarloAnisotropicBarostat(
        (pressure, pressure, pressure), temperature,  False, False, True))

The next thing we have to do is to choose the device we want to use for our simulation. In this case we select __OpenCL__.

In [ ]:
########################## Initialise GPU / CUDA / OpenCL #################
platform = mm.Platform.getPlatformByName('OpenCL')
properties = {'Precision': 'mixed'} # <-- use double for energy conservation

# platform = mm.Platform.getPlatformByName('CUDA')
# properties = {'Precision' : 'mixed' , 
#               'DeviceIndex' : '0' , 
#               'CudaCompiler' : '/usr/bin/nvcc'}

# platform = mm.Platform.getPlatformByName('CPU')
# properties = { 'Threads' : '1' }

We can now create the __simulation__ object.

In [ ]:
########################## Create simulation object #######################
simulation = mm.app.Simulation(
    pdb.topology, system, integrator, platform)

We then add the atoms' coordinates and generate their initial velocities.

In [ ]:
########################## Initialise positions and velocities ############
simulation.context.setPositions(pdb.positions)
simulation.context.setVelocitiesToTemperature(
    temperature , random.randrange(99999) )

In some instances it is good to start the simulations by doing an energy minimisation, to remove unphysical close contacts between the particles.

In [ ]:
########################## Energy minimisation ############################
if minimise:
    simulation.minimizeEnergy()

We now customise the output of out simulation. We set both a screen output and a file output.

In order to avoid overwriting the output files, we set a convention for the file names and count how many simulations we have already run in the current folder.

In [ ]:
########################## Output files ################################
nn = subprocess.getoutput("ls output.*.dat 2> /dev/null | wc -l")
ftraj   = 'trajectory.' + nn.strip() + '.dcd'    # <-- trajectory output file
fthermo = 'output.' + nn.strip() + '.dat'  # <-- output data filename

########################## Initialise the outputs #########################
# Screen output
simulation.reporters.append(
    StateDataReporter( 
        sys.stdout, int(nsteps/100), totalSteps = nsteps, separator= "\t", 
        step=False, time=True, potentialEnergy=False, kineticEnergy=False, 
        totalEnergy=False, temperature=True, volume=False, density=False, 
        progress=True, remainingTime=True, speed=True, elapsedTime=False
    )
)

# File output
simulation.reporters.append(
    StateDataReporter(
        fthermo , nthermo, separator= "\t",
        step=False, time=True, potentialEnergy=True, kineticEnergy=False, 
        totalEnergy=False, temperature=True, volume=True, density=True, 
        progress=False, remainingTime=False, speed=False, elapsedTime=False
    )
)

# Trajectory
simulation.reporters.append(
    DCDReporter( ftraj , ntraj )
)

We're finally ready to run our MD simulations

In [ ]:
simulation.step( nsteps )